<a href="https://colab.research.google.com/github/artyomsol23/google_colab/blob/main/my_homework_6_week%5Breu_ml_school%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center>**ML School by REU Data Scinece Club**</center>

----
## <center> Неделя №6 </center>

На этой неделе вам предстоит написать Дерево решений своими руками и оптимизировать его.

Импортируем библиотеки которые нам нужны

In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

Импортируем нужные данные, в этот раз мы будем пытаться предсказать ценовой диапазон цены на телефон. Данные уже обработаны

In [2]:
!git clone https://github.com/MeinRespect/REU-Ml-SCHOOL.git

Cloning into 'REU-Ml-SCHOOL'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 20 (delta 4), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (20/20), 14.94 MiB | 10.10 MiB/s, done.
Resolving deltas: 100% (4/4), done.


In [3]:
df = pd.read_csv('/content/REU-Ml-SCHOOL/mobile_price_prediction.csv')
df

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,794,1,0.5,1,0,1,2,0.8,106,6,...,1222,1890,668,13,4,19,1,1,0,0
1996,1965,1,2.6,1,0,0,39,0.2,187,4,...,915,1965,2032,11,10,16,1,1,1,2
1997,1911,0,0.9,1,1,1,36,0.7,108,8,...,868,1632,3057,9,1,5,1,1,0,3
1998,1512,0,0.9,0,4,1,46,0.1,145,5,...,336,670,869,18,10,19,1,1,1,0


Как мы видим у нас есть 4 класса, разделите данные на train и test. test_size = 0.25 и random_state = 1

In [4]:
# Ваш код тут
X, y = df.iloc[:, :-1], df.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

# **Задание 1. Функции для оптимизации (2 балла)**

Для начала реализуем критерий Gini по которому мы будем оптимизировать наше дерево. Чем данный критерий меньше, тем лучше новое разделение разделяет данные. На вход нашей функции будет подаваться массив с значениями y. Например [0, 1, 3, 2, 1, 0, 0, 0]

$$ Gini = 1 - {\sum_j}p_j^2 $$

In [20]:
def gini(labels):
    classes = {}
    for label in labels:
        if label not in classes:
            classes[label] = 0
        classes[label] += 1

    total = len(labels)
    if total == 0:
        return 0

    sum_squares = 0.0
    for count in classes.values():
        proportion = count / total
        sum_squares += proportion ** 2

    gini = 1 - sum_squares
    return gini

Проверьте правильно ли работает ваша функция. Если вы запустили код ниже и он отработал без ошибок, то значит все хорошо

In [6]:
test_cases = [
    ([0, 0, 0, 0, 0], 0.0),
    ([0, 1, 1, 1, 0], 0.48),
    ([0, 1, 2, 3, 5, 5, 2], 0.7755102040816326)
]

# Test the gini function against the test cases
for x, expected in test_cases:
    result = gini(x)
    if result == expected:
        print(f"Test case PASSED: gini({x}) = {result}")
    else:
        raise ValueError(f"Test case FAILED: gini({x}) = {result}, expected {expected}")


Test case PASSED: gini([0, 0, 0, 0, 0]) = 0.0
Test case PASSED: gini([0, 1, 1, 1, 0]) = 0.48
Test case PASSED: gini([0, 1, 2, 3, 5, 5, 2]) = 0.7755102040816326


# **Задание 2. Дерево решений (13 баллов)**

Теперь напишем сам алгоритм, в нем обязательно нужно настроить возможность ограничения дерева по глубине. Алгоритм обучения вы можете написать в удобном вам виде, главное чтобы он работал

In [23]:
class DecisionTree:
    def __init__(self, max_depth=5, min_samples_split=2):
        self.max_depth = max_depth   # задаем максимальную глубину
        self.min_samples_split = min_samples_split # миниальное число наблюдений для разбиения

    def fit(self, X, y):
        self.n_classes_ = len(np.unique(y)) # количество классов
        self.tree_ = self._plant_tree(X, y) # строим дерево

    def predict(self, X):
        return [self._predict(inputs) for inputs in X]

    def _gini(self, y):
        if len(y) == 0:
            return 0
        p = np.bincount(y) / len(y)
        return 1 - np.sum(p ** 2)

    # Функция которая поможет найти лучший признак для разбиения в наших данных
    def _best_split(self, X, y):
        best_idx, best_thr = None, None
        best_gini = float('inf')
        num_of_y = y.size # Сразу проверим что у нас больше чем одно наблюдение
        if num_of_y <= 1:
            return None, None

        # Создадим list, где у нас будет указано количество каждого класса
        num_of_each_class = [np.sum(y == c) for c in range(self.n_classes_)]

        # Ваш код тут, вам нужно:
        # 1) Вернуть индекс колонки где лучше всего разбить данные
        # 2) Найти в этой колоники лучшее значения для разбиения и вернуть их
        # Также учитывайте, что если после рзабиения в одном из листьев у вас меньше
        # чем min_samples_split, вам не подходит такое разбиение

        for idx in range(X.shape[1]):
          thresholds = np.unique(X[:, idx])

          for thr in thresholds:
            left_mask = X[:, idx] < thr
            right_mask = ~left_mask

            if (np.sum(left_mask) < self.min_samples_split or
                np.sum(right_mask) < self.min_samples_split):
                  continue

        # Важный момент, при подсчете gini после разбиения, нужно подсчитать его как для
        # правого листа, так и для левого, а затем расччитать общий gini.
        # gini = (i * gini_left + (m - i) * gini_right) / m
        # i - количество наблюдений в левом листе, m - в правом

            gini_left = self._gini(y[left_mask])
            gini_right = self._gini(y[right_mask])
            total_gini = (np.sum(left_mask) * gini_left +
                          np.sum(right_mask) * gini_right) / num_of_y

            if total_gini < best_gini:
              best_gini = total_gini
              best_idx = idx
              best_thr = thr

        return best_idx, best_thr

    def _plant_tree(self, X, y, depth=0):

        # Для начала мы предскажем класс, взяв максимальное тот, который чаще всего встречается в данных
        num_samples_per_class = [np.sum(y == i) for i in range(self.n_classes_)]
        predicted_class = np.argmax(num_samples_per_class)

        # Тут нам будет помогать другой класс Node, по-другому лист.
        # Мы будем использовать его для сохранения информации о разбиении
        node = Node(predicted_class=predicted_class)

        if depth < self.max_depth:
            idx, thr = self._best_split(X, y)
            if idx is not None:

                # Ваш код тут. Создайте переменные с значениями X и y которые отправлись в правый лист.
                # создайте также переменные с значениями, которые отправились в левый лист
                # Другими словами просто разделите данные на две части по вашему best_split

                left_mask = X[:, idx] < thr
                right_mask = ~left_mask

                X_left, y_left = X[left_mask], y[left_mask]
                X_right, y_right = X[right_mask], y[right_mask]

                node.feature_index = idx
                node.threshold = thr

                node.left = self._plant_tree(X_left, y_left, depth + 1)  # вызовите plant_tree и передайте туда ваши переменные в левой части, к глубине прибавьте +1
                node.right = self._plant_tree(X_right, y_right, depth + 1)  # вызовите plant_tree и передайте туда ваши переменные в правой части, к глубине прибавьте +1

        return node

    def _predict(self, inputs):
        node = self.tree_
        while node.left:
            if inputs[node.feature_index] < node.threshold:
                node = node.left
            else:
                node = node.right
        return node.predicted_class

class Node:
    def __init__(self, *, predicted_class):
        self.predicted_class = predicted_class
        self.feature_index = 0
        self.threshold = 0
        self.left = None
        self.right = None


Теперь протестируйте свое дерево и покажите результат f1_score(укажите в нем average = 'marco') для 2 видов глубины, на свое усмотрение

In [25]:
# Ваш код здесь
X_train_np, y_train_np = X_train.values, y_train.values
X_test_np, y_test_np = X_test.values, y_test.values

tree_depth_3 = DecisionTree(max_depth=3, min_samples_split=2)
tree_depth_10 = DecisionTree(max_depth=10, min_samples_split=2)

tree_depth_3.fit(X_train_np, y_train_np)
tree_depth_10.fit(X_train_np, y_train_np)

y_pred_3 = tree_depth_3.predict(X_test_np)
y_pred_10 = tree_depth_10.predict(X_test_np)

f1_3 = f1_score(y_test_np, y_pred_3, average='macro')
f1_10 = f1_score(y_test_np, y_pred_10, average='macro')

print(f'f1_3 = {f1_3:.4f}')
print(f'f1_10 = {f1_10:.4f}')

f1_3 = 0.7374
f1_10 = 0.8447


# **Задание 3. Деревья в Sklearn (5 баллов)**

Напишите поиск параметров(их должно быть минимум 4) с помощью GridSearchCV для RandomForestClassifier. Он может долго перебирать параметры, это нормально. Оцените результат с помощью f1_score, укажите в нем average = 'marco'

In [28]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

rf = RandomForestClassifier(random_state=42)

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2'],
    'bootstrap': [True, False]
}

scorer = make_scorer(f1_score, average='macro')

grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    scoring=scorer,
    cv=5,
    n_jobs=-1,
    verbose=2
)

grid_search.fit(X_train, y_train)

print("Лучшие параметры:", grid_search.best_params_)

best_rf = grid_search.best_estimator_
y_pred = best_rf.predict(X_test)
test_f1 = f1_score(y_test, y_pred, average='macro')

print(f"F1-score (macro) на тестовых данных: {test_f1:.4f}")

Fitting 5 folds for each of 432 candidates, totalling 2160 fits
Лучшие параметры: {'bootstrap': True, 'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 200}
F1-score (macro) на тестовых данных: 0.8645
